In [88]:
#CodeMusai Alpha - 0.0.2

#Create a concept, memoryspan/blockSize that defines awareness context when making a prediction.
#The last model was always 2 due to the data-structure, this enhncement will add meta-parameters.

#These results will be a lot better then the last, but due to model simplicity lowish quality 
#results are still expected.

In [89]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt #diagrams
%matplotlib inline

words = open('activeTrainingMaterial_lower.txt', 'r').read().lower().split()
words[:8]

['a', 'painted', 'christmas', 'an', 'original', 'story', 'of', 'life']

In [90]:
len(words)

156969

In [104]:
#config for later
block_size = 3 #context for prediction
numNeurons = 10
nembd = 16
batch_size = 32

def configure(theBlockSize, theNumNeurons, theNembd, theBatchSize):#32,16,10,3
    global block_size, numNeurons, nembd, batch_size
    block_size = theBlockSize
    numNeurons = theNumNeurons
    nembd = theNembd
    batch_size = theBatchSize 

#build vocabulary
chars = sorted(list(set(''.join(words))))
print(chars)
characterCount = len(chars)+1
print(f"{characterCount} chars found")
stoi = {s:i+1 for i,s in enumerate(chars)}
#spotIdx = stoi['.']
#spotEndIdx = stoi['\'']
#stoi['.'] = 0
#stoi['\''] = spotIdx
itos = {i:s for s,i in stoi.items()}
print(itos)
ix = 0

['.', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
30 chars found
{1: '.', 2: ';', 3: '?', 4: 'a', 5: 'b', 6: 'c', 7: 'd', 8: 'e', 9: 'f', 10: 'g', 11: 'h', 12: 'i', 13: 'j', 14: 'k', 15: 'l', 16: 'm', 17: 'n', 18: 'o', 19: 'p', 20: 'q', 21: 'r', 22: 's', 23: 't', 24: 'u', 25: 'v', 26: 'w', 27: 'x', 28: 'y', 29: 'z'}


In [105]:
#build the dataset
def buildDataset(words):    
    print('Building the Dataset...');
    #block_size = ...
    X, Y = [], []
    for w in words:
        #print(w)
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            #print(''.join(itos[i] for i in context), '---->', itos[ix])
            context = context[1:] + [ix] #crop and append
            
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(327)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = buildDataset(words[:n1])
Xdev, Ydev = buildDataset(words[n1:n2])
Xte, Yte = buildDataset(words[n2:])

Building the Dataset...
torch.Size([774053, 3]) torch.Size([774053])
Building the Dataset...
torch.Size([95662, 3]) torch.Size([95662])
Building the Dataset...
torch.Size([96416, 3]) torch.Size([96416])


In [106]:
#generate       (defined for later)
def generate(numSamples):
    print(f"Generating {numSamples}...")
    for _ in range(numSamples):
        out = []
        context = [0] * block_size
        while True:
            emb = C[torch.tensor([context])] #(1,block_size,d)
            h = torch.tanh(emb.view(1, -1) @ W1 + b1)
            logits = h @ W2 + b2
            probs = F.softmax(logits, dim=1)
            ix = torch.multinomial(probs, num_samples=1, replacement=True).item() #, replacement=True)
            context = context[1:] + [ix]
            out.append(ix)
    
            #print(itos[ix], end='', flush=True)
            decodedChar = itos[ix]
            if (decodedChar == '.'):
                decodedChar = ' '
                
            print(decodedChar, end='', flush=True)
            
            if ix == 0:
                break
                
    print('.', end='', flush=True)#print(''.join(out) + '.')   
        #output = ''.join(itos[i] for i in out).replace('.', ' ')
        #print(output, end='', flush=True)

In [107]:

def initializeWeights():
    print('Initializing Weights...')
    #initialize weights, and prepare for training
    #~
    
    #numNeurons = ...
    #nembd = ...
    #-
    #emb = C[Xtr[ix]]  # Should be [32, 3, 2] # Check if elements align before reshaping
    #total_elements = emb.numel()  # Should be 32 * 3 * 2 = 192
    #new_shape = (-1, 3 * nembd)  # Should lead to (-1, 6) ensuring 192 elements fit
    #-
    C = torch.randn((characterCount+1, nembd))
    
    W1 = torch.randn((3*nembd,numNeurons), requires_grad=True)
    b1 = torch.randn(numNeurons, requires_grad=True)
    
    W2 = torch.randn((numNeurons, characterCount), requires_grad=True)
    b2 = torch.randn(characterCount, requires_grad=True)
    
    parameters = [C, W1, b1, W2, b2]
    
    for p in parameters:
        p.requires_grad = True
    
    lre = torch.linspace(-3, 0, 1000) #0.001 -> -3,    1 -> 0
    lrs = 10**lre
    
    
    numParams = sum(p.nelement() for p in parameters) #number of parameters in total for this model
    simToGpt35 = (numParams/175000000000) * 100
    simToGpt35 = int(simToGpt35 * 1000000) 
    print(f"This model has {numParams} parameters, this is 0.00000{simToGpt35}% of gpt3.5")
    return parameters

In [108]:
def trainingLoop(trainingIterations, lr, autoPlot):
    print(f"Starting Training of {trainingIterations} at a learning rate of {lr}...")
    lri = []
    lossi = []
    stepi = []
    #autoPlot = False
    #batch_size = ...
    for i in range(trainingIterations):
    
        if autoPlot:
            if (i > 0 and batch_size % i == 0):
                plt.figure(figsize=(8,8))
                plt.scatter(C[:, 0].data, C[:,1].data, s=200)
                for i in range(C.shape[0]):
                    plt.text(C[i,0].item(), C[i,1].item(), itos[i], ha="center", va="center", color='white')
                plt.grid('minor')
    
        #minibatch construct
        ix = torch.randint(0, Xtr.shape[0], (batch_size,))
        #print(C[Xtr[ix]])

        
        #forward pass
        emb = C[Xtr[ix]] #[32, 3, 2)
        
        h = torch.tanh(emb.view(-1, block_size*nembd) @ W1 + b1) # (32, 100)
        logits = h @ W2 + b2 # (32, characterCount)
        #print(f"Logits shape: {logits.shape}")
        #print(f"Target shape: {Ytr[ix].shape}") 
        loss = F.cross_entropy(logits, Ytr[ix]) #IT MUST USE CROSS_ENTROPY
        #print(loss.item())
        
        
        
        #backward pass
        for p in parameters:
            if not p.requires_grad:
                print(f"Parameter {p} does not require gradients.")
            p.grad = None
        loss.backward() 
        
        #update model
        #lr = lrs[i]
        #lr = 0.1 #initial version
        #lr = 0.01 #decay version - after initial
        for p in parameters:
            if p.grad is not None:
                p.data += -lr * p.grad
    
        #track stats  to deterine learning rate
        #lri.append(lre[i])
        #lossi.append(loss.item())
        stepi.append(i)
        lossi.append(loss.log10().item())
    
    print('Loss: ', loss.item())

In [109]:
def lossFnc(type):
    print(f"Calculating {type} Loss...")
    if type == 'dev':
        emb = C[Xdev] #(32, 3, 2)
        h = torch.tanh(emb.view(-1, 3*nembd ) @ W1 + b1) #(32,100)
        logits = h @ W2 + b2
        loss = F.cross_entropy(logits, Ydev)
        loss
    elif type == 'train': #train loss
        emb = C[Xtr] #(32, 3, 2)
        h = torch.tanh(emb.view(-1, 3*nembd ) @ W1 + b1) #(32,100)
        logits = h @ W2 + b2
        loss = F.cross_entropy(logits, Ytr)
        loss
    elif type == 'val': #test loss - use sparingly
        emb = C[Xte] #(32, 3, 2)
        h = torch.tanh(emb.view(-1, 3*nembd ) @ W1 + b1) #(32,100)
        logits = h @ W2 + b2
        loss = F.cross_entropy(logits, Yte)
    
    print(f"Loss for {type}: ", loss)
    


In [ ]:
#Main Running Entry Point
init = False
3,10,16,32
configure(theBlockSize=3, theNumNeurons=100, theNembd=32, theBatchSize=64)
#32,768,10,64
if not init:
    [C, W1, b1, W2, b2] = initializeWeights()
    parameters = [C, W1, b1, W2, b2]
    init = True
    
#pretraining
trainingLoop(trainingIterations=50000, lr=0.1, autoPlot=False)
trainingLoop(trainingIterations=25000, lr=0.07, autoPlot=False)
trainingLoop(trainingIterations=5000, lr=0.05, autoPlot=False)

lossFnc('dev')
lossFnc('train')
print('')
generate(numSamples=100)

Initializing Weights...
This model has 13722 parameters, this is 0.000007% of gpt3.5
Starting Training of 50000 at a learning rate of 0.1...
Loss:  1.4945532083511353
Starting Training of 25000 at a learning rate of 0.07...
Loss:  1.5615580081939697
Starting Training of 5000 at a learning rate of 0.05...
Loss:  1.8960307836532593
Calculating dev Loss...
Loss for dev:  tensor(1.6101, grad_fn=<NllLossBackward0>)
Calculating train Loss...
Loss for train:  tensor(1.6027, grad_fn=<NllLossBackward0>)

Generating 100...
in ns                                                                                                                                                                                                                                                                                                                                                                                                                                                                             y              

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



                                                                                                                                          plack                                                                                                                                           y                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      eprescosl                                                                                                                                                                                            t                                                                                                                                                                                                            

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



                                                                                                                                                           ss                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        sshdieus                                                                                                                                                                                                                                                                                                                                   pnall                                   